In [1]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Invalid requirement: 'rags=0.1.11': Expected end or semicolon (after name and no valid version specifier)
    rags=0.1.11
        ^ (from line 14 of requirements.txt)
Hint: = is not a valid operator. Did you mean == ?


In [2]:
import gradio as gr
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DESCRIPTION = '''
<div>
<h1 style="text-align: center;">EmoLLM 心理咨询室</h1>

<p align="center">
  <a href="https://github.com/Rvosuke/EmoLLM/">
  </a>
</p>

</div>
'''

LICENSE = """
<p align="center"> Built with 26组 白泽阳 & 钱兴宇 </>
"""

PLACEHOLDER = """
<div style="padding: 30px; text-align: center; display: flex; flex-direction: column; align-items: center;">

</div>
"""


css = """
h1 {
  text-align: center;
  display: block;
}
<!--
#duplicate-button {
  margin: auto;
  color: white;
  background: #1565c0;
  border-radius: 100vh;
}
-->
"""

In [5]:
# download internlm2 to the base_path directory using git tool
base_path = os.path.expandvars('$GEMINI_PRETRAIN2')
# os.system(f'git clone https://code.openxlab.org.cn/chg0901/EmoLLM-Llama3-8B-Instruct3.0.git {base_path}')
# os.system(f'cd {base_path} && git lfs pull')


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_path,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_path,trust_remote_code=True, device_map="auto", torch_dtype=torch.float16).eval()  # to("cuda:0") 
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def chat_llama3_8b(message: str, 
              history: list, 
              temperature: float, 
              max_new_tokens: int,
              top_p: float
             ) -> str:
    """
    Generate a streaming response using the llama3-8b model.
    Args:
        message (str): The input message.
        history (list): The conversation history used by ChatInterface.
        temperature (float): The temperature for generating the response.
        max_new_tokens (int): The maximum number of new tokens to generate.
    Returns:
        str: The generated response.
    """
    conversation = []
    message += "你是一个心理咨询师，请使用中文来解决各种心理健康问题。注意循序渐进地回复，一次不要回复太多。"
    
    for user, assistant in history:
        conversation.extend([{"role": "user", "content": user}, {"role": "assistant", "content": assistant}])
    conversation.append({"role": "user", "content": message})

    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)
    
    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    generate_kwargs = dict(
        input_ids= input_ids,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p = top_p,
        eos_token_id=terminators,
    )
    # This will enforce greedy generation (do_sample=False) when the temperature is passed 0, avoiding the crash.             
    if temperature == 0:
        generate_kwargs['do_sample'] = False
        
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    outputs = []
    for text in streamer:
        outputs.append(text)
        yield "".join(outputs)

        

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:52<00:00, 13.04s/it]


In [6]:
# Gradio block
# , placeholder=PLACEHOLDER
chatbot=gr.Chatbot(height=450, label='EmoLLM Chat')

with gr.Blocks(css=css) as demo:
    
    gr.Markdown(DESCRIPTION)
    # gr.DuplicateButton(value="Duplicate Space for private use", elem_id="duplicate-button")
    gr.ChatInterface(
        fn=chat_llama3_8b,
        chatbot=chatbot,
        additional_inputs_accordion=gr.Accordion(label="⚙️ Parameters", open=False, render=False),
        additional_inputs=[
            gr.Slider(minimum=0,
                      maximum=1, 
                      step=0.1,
                      value=0.95, 
                      label="Temperature", 
                      render=False),
            gr.Slider(minimum=128, 
                      maximum=4096,
                      step=1,
                      value=4096, 
                      label="Max new tokens", 
                      render=False ),
            gr.Slider(minimum=0.0, 
                      maximum=1,
                      step=0.01,
                      value=0.8, 
                      label="Top P", 
                      render=False ),
            # gr.Slider(minimum=128, 
            #           maximum=4096,
            #           step=1,
            #           value=512, 
            #           label="Max new tokens", 
            #           render=False ),
            ],
        examples=[
            ['请介绍你自己。'],
            ['我觉得我在学校的学习压力好大啊，虽然我真的很喜欢我的专业，但最近总是担心自己无法达到自己的期望，这让我有点焦虑。'],
            ['我最近总觉得自己在感情上陷入了困境，我喜欢上了我的朋友，但又害怕表达出来会破坏我们现在的关系...'],
            ['我感觉自己像是被困在一个无尽的循环中。每天醒来都感到身体沉重，对日常活动提不起兴趣，工作、锻炼甚至是我曾经喜欢的事物都让我觉得厌倦'],
            ['最近工作压力特别大，还有一些家庭矛盾']
            ],
        cache_examples=False,
                     )
    
    gr.Markdown(LICENSE)
    

In [7]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.7.1, however version 4.29.0 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 4.7.1, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://ccd93ccffa96fdcb35.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/root/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass